In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

In [3]:
training_data = torchvision.datasets.MNIST('/files/', train=True, download=True, transform=torchvision.transforms.ToTensor())
test_data = torchvision.datasets.MNIST('/files/', train=False, download=True, transform=torchvision.transforms.ToTensor())

In [6]:
n_epochs = 10
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.9
log_interval = 100

In [7]:
train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size_test, shuffle=True)

In [8]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
image_dimensions = example_data[0][0].size()

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)
        self.sigmoid1 = nn.Sigmoid()
        self.avg1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.sigmoid2 = nn.Sigmoid()
        self.avg2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16*5*5, 120)
        self.sigmoid3 = nn.Sigmoid()
        self.fc2 = nn.Linear(120, 84)
        self.sigmoid4 = nn.Sigmoid()
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.sigmoid1(x)
        x = self.avg1(x)
        x = self.conv2(x)
        x = self.sigmoid2(x)
        x = self.avg2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.sigmoid3(x)
        x = self.fc2(x)
        x = self.sigmoid4(x)
        x = self.fc3(x)
        return x


In [10]:
net = Net()
# net.cuda()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

In [11]:
train_losses = []
train_counter = []
test_losses = []

In [13]:
def train(epoch):
    net.train()
    for batch_idx, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        y_hat = net(x)
        l = loss(y_hat, y)
        l.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), l.item()))
            train_losses.append(l.item())
            train_counter.append(
                (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [14]:
def test():
  net.eval()
  test_loss = 0
  correct = 0
  test_lenth = 0
  with torch.no_grad():
    for x, y in test_loader:
      # x = x.cuda()
      # y = y.cuda()
      y_hat = net(x)
      test_loss +=  loss(y_hat, y).item()
      test_lenth += 1
      pred = y_hat.data.max(1, keepdim=True)[1]
      correct += pred.eq(y.data.view_as(pred)).sum()
  test_loss /= test_lenth
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [26]:
for epoch in range(0, n_epochs):
  train(epoch)
  test()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.329002
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.286155
Train Epoch: 0 [12800/60000 (21%)]	Loss: 2.280570
Train Epoch: 0 [19200/60000 (32%)]	Loss: 2.297360
Train Epoch: 0 [25600/60000 (43%)]	Loss: 2.307328
Train Epoch: 0 [32000/60000 (53%)]	Loss: 2.292676
Train Epoch: 0 [38400/60000 (64%)]	Loss: 2.301775
Train Epoch: 0 [44800/60000 (75%)]	Loss: 2.311617
Train Epoch: 0 [51200/60000 (85%)]	Loss: 2.300071
Train Epoch: 0 [57600/60000 (96%)]	Loss: 2.294806

Test set: Avg. loss: 0.0023, Accuracy: 1135/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300382
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.293246
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.304957
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.301935
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.307050
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.303960
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.288554
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.301531
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.291297
Trai